In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np

sns.set()
warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('../final_data/combined_data.csv')

In [3]:
movies.sample(2)

,belongs_to_collection,budget,genres,movie_id,original_language,overview,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,production_countries_code,release_year,release_month,release_day,return,director,actors
33948,NaN,0.0,Thriller,67810,en,Hard is about a gay serial killer and the gay ...,0.082495,NaN,NaN,1953-06-24,...,Hard,5.5,2.0,NaN,1953,6,24,0.0,John Huckert,Noel Palomaria|Malcolm Moorman|Charles Lanyer|...
41555,NaN,0.0,Comedy|Drama|Romance,320293,fi,"The main character is a miller, who sometimes ...",0.070588,Filmityö,Finland,2016-12-08,...,The Howling Miller,6.0,2.0,FI,2016,12,8,0.0,Jaakko Pakkasvirta,Vesa-Matti Loiri|Eija Ahvo|Yrjö Järvinen|Harri...


#### Seleccion de las columnas
*No selecciono la columna de overview porque si no mi matriz de similitud se va hacer muy pesada y dificil de procesar*

In [4]:
movies = movies[['title','genres','director']]
movies.sample(2)

,title,genres,director
27246,New World Order,Documentary,Luke Meyer
19642,Tom & Thomas,Drama|Family,Esmé Lammers


In [5]:
movies.genres = movies.genres.str.replace('|',' ')

In [6]:
movies['joined_data'] = movies[['title','genres','director']].astype(str).apply(' '.join, axis=1)
movies.sample()

,title,genres,director,joined_data
42147,Stolen Death,Thriller,Nyrki Tapiovaara,Stolen Death Thriller Nyrki Tapiovaara


Cálculo de similitud de puntuación


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
 
# Elejí usar TfidfVectorizer para la vectorizacion de los datos en vez de CountVectorizer. Si bien este ultimo es mejor por cuestiones de memoria, 
# el TfidfVectorizer brinda mayor precision a la hora de buscar una puntuación de similitud en las cadenas de strings

In [8]:
vectorizer = TfidfVectorizer(stop_words='english') # Elimino las palabras mas comunes del ingles

In [9]:
def get_recommendation(title):
    """Get the recommendation for a given title"""
    # Obtengo los géneros de la pelicula
    movie_genres = movies['genres'].loc[movies['title'] == title]

    # Separo los generos en una lista
    genres = [v.split(' ') for v in movie_genres if isinstance(v,str)]

    # Filtro el dataframe con las peliculas que tengan los mismos géneros que la pelicula de interes
    # y con las peliculas que tengan en su titulo el nombre de la pelicula de interes
    similar_movies = movies.loc[movies.genres.fillna(' ').str.contains(movie_genres.values[0]) | movies.title.str.contains(title)].reset_index(drop=True)
   
    # Vectorizo los datos
    data_matrix = vectorizer.fit_transform(similar_movies.joined_data)
    # Genero la matriz de puntuación de similitud entre las peliculas
    similarity_matrix = cosine_similarity(data_matrix,data_matrix)

    # Obtengo el indice en el dataframe filtrado de la pelicula de interes
    movie_index = similar_movies.loc[similar_movies['title'] == title].index 

    sim_movies = list(enumerate(similarity_matrix[movie_index[0]]))
    sim_movies = sorted(sim_movies, key=lambda x: x[1], reverse=True)
    top_similar_movies = [similar_movies.loc[i, 'title'] for i, _ in sim_movies[1:20] if similar_movies.loc[i, 'title'] != title][:5]
    return top_similar_movies

In [30]:
get_recommendation('Cars')

['Cars 2', "A Bug's Life", 'Cars 3', 'The Ant Bully', 'Dino Time']